In [1]:
import os.path 
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd 
from numpy import ndarray

# Folder where the created images will be saved in
out_path = r'/local/data1/elech646/Tumor_grade_classification/single_channel_slices/slices_t1_flair'
dataset_path = r'/local/data1/elech646/Tumor_grade_classification/original_dataset/LGG'

# Create subfolders
if not os.path.exists(out_path + "/sagittal_grade_classification"):
    os.mkdir(out_path + "/sagittal_grade_classification")
    
if not os.path.exists(out_path + "/frontal_grade_classification"):
    os.mkdir(out_path + "/frontal_grade_classification")

if not os.path.exists(out_path + "/trans_grade_classification"):
    os.mkdir(out_path + "/trans_grade_classification")

# Add LGG path
sag_path = out_path + "/sagittal_grade_classification" + "/LGG"
fro_path = out_path + "/frontal_grade_classification" + "/LGG"
tra_path = out_path + "/trans_grade_classification" + "/LGG"
    
if not os.path.exists(sag_path):
    os.mkdir(sag_path)
    
if not os.path.exists(fro_path):
    os.mkdir(fro_path)
    
if not os.path.exists(tra_path):
    os.mkdir(tra_path)

In [4]:
seg_path = []
patient_name = []

np.seterr(divide = 'ignore', invalid = 'ignore')

for roots, dirs, files in os.walk("/local/data1/elech646/OUTPUT_DIRECTORY"):
    for name in files:
        if name.endswith((".nii.gz", ".nii")):
            seg_path.append(roots + os.path.sep + name)
            patient_name.append(name.split('.')[0])

# Import Excel file
annotations_LGG = pd.read_excel(r'/local/data1/elech646/Tumor_grade_classification/LGG case annotations.xlsx')

# Keep only patient id + manual tumor coordinates
annotations_LGG = annotations_LGG[annotations_LGG.Segmented == 'NO'].reset_index(drop = True).\
drop(["CDE_TYPE", "Laterality", "Segmented"], axis = 1)
            
# Loop through the subjects
for p_name, s_path in zip(patient_name, seg_path):
    # Load the segmentation file
    seg_img = nib.load(s_path)
    seg_img_data = seg_img.get_fdata()
    
    # Create subfolders 
    if not os.path.exists(os.path.join(sag_path, p_name)):
        os.mkdir(os.path.join(sag_path, p_name))
    if not os.path.exists(os.path.join(fro_path, p_name)):
        os.mkdir(os.path.join(fro_path, p_name))
    if not os.path.exists(os.path.join(tra_path, p_name)):
        os.mkdir(os.path.join(tra_path, p_name))
    
    # Loop through the modalities of choice
    modalities = ['t1', 'flair']
    aus_modalities = []
                 
    for m in modalities:
        #print(f'Working on {p_name}, modality {m} \r', end = '')
        # Load full image for this modality
        #mod_img = nib.load(os.path.join(dataset_path, patient_name[i], patient_name[i] + '_' + idx + '.nii'))
        mod_img = nib.load(os.path.join(dataset_path, p_name, '_'.join([p_name, m]) + '.nii.gz'))
        mod_img_data = mod_img.get_fdata()
        
        # Get min, max
        min_v = mod_img_data.min()
        max_v = mod_img_data.max()

        # Normalize the image
        mod_img_data = (255*(mod_img_data - min_v) / (max_v - min_v)).astype(np.uint8)
        aus_modalities.append(mod_img_data)
        
############################ Uncomment for the function you want every time ######################################

    # Start creating images for the sagittal plane
    def create_sagittal(list_modality, 
                        segmentation, 
                        save_path = None, 
                        from_excel = False, 
                        sag_0 = None, 
                        sag_1 = None):
        '''
        Creates sagittal images as RGB stacking tumor slices from the 
        given modalities as individual channels
        
        Arguments:
        list_modality : list
            Each element coming from something like mod_img.get_fdata()
        segmentation : np array
            binary volume defining where the tumor is
        save_path : str
            the path where images are saved        
        from_excel : boolean
            whether the images should be checked from an Excel or not
        sag_0 : float
            if the images are read from NIfTI files then get min tumor index from there,
            otherwise read zmin from Excel file
        sag_1 : float
            if the images are read from NIfTI files then get max tumor index from there,
            otherwise read zmax from Excel file
        '''
        
        if not from_excel:
            # fix the volumes to match the sagittal plane
            # segmentation
            segmentation = np.rot90(segmentation, axes = (1, 2)) # yz plane sagittal
            segmentation = np.flip(segmentation, 0) # flip the image left/right 

            # the modality volumes
            for idx, v in enumerate(list_modality):
                # do the magic for Mango
                v = np.rot90(v, axes = (1, 2)) # yz plane sagittal
                v = np.flip(v, 0) # flip the image left/right
                list_modality[idx] = v

            # Get indices of tumor slices
            sag_0 = min(ndarray.nonzero(segmentation)[0])   # zmin
            sag_1 = max(ndarray.nonzero(segmentation)[0])   # zmax
        
        # Loop through all the indices
        for sag in range(sag_0, sag_1 + 1):
            try:
                perc = int(((sag - sag_0)/(sag_1 - sag_0))*100) # Percentage along the selected slices
            except ValueError:
                perc = 'NA'
            
            aus_mod = []
            for idx in range(len(list_modality)):
                aus_mod.append(list_modality[idx][sag,:,:])
            
            # check that we have all the channels
            if len(aus_mod) != 3:
                for i in range(3-len(aus_mod)):
                    aus_mod.append(aus_mod[-1])
                    
            # convert list to array [W, H, CH]
            slices = np.stack(aus_mod, axis = -1)
                         
            # name the files
            aus_name = modalities
            if len(modalities) != 3:
                for i in range(3-len(modalities)):
                    aus_name.append(modalities[-1])
            aus_name = '_'.join(aus_name)
                
            title = os.path.join(save_path, p_name, p_name + '_sag_' + aus_name +\
                                 '_' + str(sag) + '_' + str(perc) + '.png')
            
            # convert to RGB
            im = Image.fromarray(slices).convert('RGB')
            
            # save images
            im.save(title)
    
    # Tweaking here since we have segmented files with 0% tumor
    try:
        create_sagittal(aus_modalities, seg_img_data, save_path = sag_path) 
    except:
        # Map patient names 
        index_patient = [i for i, v in enumerate(annotations_LGG.itertuples()) if v.CDE_ID == p_name]
        
        # Get indices of tumor slices   
        zmin = int(annotations_LGG.at[index_patient[0], 'zmin'])   
        zmax = int(annotations_LGG.at[index_patient[0], 'zmax'])   
        
        patient_id = annotations_LGG.CDE_ID.map(str)
        patient_id = patient_id.tolist()
        
        create_sagittal(aus_modalities, seg_img_data, save_path = sag_path, 
                        from_excel = True, 
                        sag_0 = zmin, 
                        sag_1 = zmax) 
        
    # Start creating images for the frontal plane
#     def create_frontal(list_modality, 
#                        segmentation, 
#                        save_path = None, 
#                        from_excel = False, 
#                        fr_0 = None, 
#                        fr_1 = None):
#         '''
#         Creates frontal images as RGB stacking tumor slices from the 
#         given modalities as individual channels.
        
#         Arguments: 
#         list_modality : list
#             Each element coming from something like mod_img.get_fdata()
#         segmentation : np array
#             binary volume defining where the tumor is
#         save_path : str
#             the path where images are saved  
#         from_excel : boolean
#             whether the images should be checked from an Excel or not
#         fr_0 : float
#             if the images are read from NIfTI files then get min tumor index from there,
#             otherwise read ymin from Excel file
#         fr_1 : float
#             if the images are read from NIfTI files then get max tumor index from there,
#             otherwise read ymax from Excel file
#         '''
#         if not from_excel:
#             # fix the volumes to match the frontal plane
#             # segmentation
#             segmentation = np.rot90(segmentation, axes = (0,2)) # xz plane frontal

#             # the modality volumes
#             for idx, v in enumerate(list_modality):
#                 # do the magic for Mango
#                 v = np.rot90(v, axes = (0,2))
#                 list_modality[idx] = v

#             # Get indices of tumor slices
#             fr_0 = min(ndarray.nonzero(segmentation)[1])   # ymin
#             fr_1 = max(ndarray.nonzero(segmentation)[1])   # ymax

#         # Loop through all the indices
#         for front in range(fr_0, fr_1 + 1):
#             try:
#                 perc = int(((front - fr_0) /(fr_1 - fr_0))*100) # Percentage along the selected slices
#             except ValueError:
#                 perc = 'NA'
            
#             aus_mod = []
#             for idx in range(len(list_modality)):
#                 aus_mod.append(list_modality[idx][:,front,:])
            
#             # check that we have all the channels
#             if len(aus_mod) != 3:
#                 for i in range(3-len(aus_mod)):
#                     aus_mod.append(aus_mod[-1])
                    
#             # convert list to array [W, H, CH]
#             slices = np.stack(aus_mod, axis = -1)
                         
#             # Name the files
#             aus_name = modalities
#             if len(modalities) != 3:
#                 for i in range(3-len(modalities)):
#                     aus_name.append(modalities[-1])
#             aus_name = '_'.join(aus_name)
                
#             title = os.path.join(save_path, p_name, p_name + '_fro_' + aus_name +\
#                                  '_' + str(front) + '_' + str(perc) + '.png')
#             # Convert to RGB
#             im = Image.fromarray(slices).convert('RGB')
            
#             # Save images
#             im.save(title)
            
#     # Tweaking here since we have segmented files with 0% tumor
#     try:
#         create_frontal(aus_modalities, seg_img_data, save_path = fro_path) 
#     except ValueError:
#         # Map patient names 
#         index_patient = [i for i, v in enumerate(annotations_LGG.itertuples()) if v.CDE_ID == p_name]
        
#         # Get indices of tumor slices      
#         ymin = int(annotations_LGG.at[index_patient[0], 'ymin'])    
#         ymax = int(annotations_LGG.at[index_patient[0], 'ymax'])  
        
#         patient_id = annotations_LGG.CDE_ID.map(str)
#         patient_id = patient_id.tolist()
        
#         create_frontal(aus_modalities, seg_img_data, save_path = fro_path, 
#                        from_excel = True, 
#                        fr_0 = ymin, 
#                        fr_1 = ymax)   

    # Start creating images for the transversal plane
#     def create_transversal(list_modality, 
#                            segmentation, 
#                            save_path = None,
#                            from_excel = False,
#                            tr_0 = None,
#                            tr_1 = None):
#         '''
#         Creates transversal images as RGB stacking tumor slices from the 
#         given modalities as individual channels.
        
#         Arguments: 
#         list_modality : list
#             Each element coming from something like mod_img.get_fdata()
#         segmentation : np array
#             binary volume defining where the tumor is
#         save_path : str
#             the path where images are saved  
#         from_excel : boolean
#             whether the images should be checked from an Excel or not
#         tr_0 : float
#             if the images are read from NIfTI files then get min tumor index from there,
#             otherwise read xmin from Excel file
#         tr_1 : float
#             if the images are read from NIfTI files then get max tumor index from there,
#             otherwise read xmax from Excel file
#         '''
#         if not from_excel:
#             # fix the volumes to match the transversal plane
#             # segmentation
#             segmentation = np.rot90(segmentation, 3, axes = (0,1)) # xy plane transversal
#             x, y, z = segmentation.shape
            
#             # the modality volumes
#             for idx, v in enumerate(list_modality):
#                 # do the magic for Mango
#                 v = np.rot90(v, 3, axes = (0,1))
#                 list_modality[idx] = v

#             # Get indices of tumor slices
#             # Tweaking here since we have segmented files with 0% tumor
#             tr_1 = max(ndarray.nonzero(segmentation)[2])  # xmax
#             tr_0 = min(ndarray.nonzero(segmentation)[2])  # xmin

#         if from_excel:
#             # Inverting slices upside/down since mango was used
#             segmentation = np.rot90(segmentation, 3, axes = (0,1)) # xy plane transversal
#             x, y, z = segmentation.shape
            
#             tr_0 = z - tr_0
#             tr_1 = z - tr_1
        
#         # loop through all the indices
#         for transv in range(tr_0, tr_1):
#             try:
#                 # Percentage along the selected slices
#                 perc = int(((transv - tr_0) /(tr_1 - tr_0))*100) 
#             except ValueError:
#                 perc = 'NA'
            
#             aus_mod = []
#             for idx in range(len(list_modality)):
#                 aus_mod.append(list_modality[idx][:,:, transv])
            
#             # Check that we have all the channels
#             if len(aus_mod) != 3:
#                 for i in range(3-len(aus_mod)):
#                     aus_mod.append(aus_mod[-1])
                    
#             # Convert list to array [W, H, CH]
#             slices = np.stack(aus_mod, axis = -1)
                         
#             # Name the files
#             aus_name = modalities
#             if len(modalities) != 3:
#                 for i in range(3-len(modalities)):
#                     aus_name.append(modalities[-1])
#             aus_name = '_'.join(aus_name)
                
#             title = os.path.join(save_path, p_name, p_name + '_trans_' + aus_name +\
#                                  '_' + str(transv) + '_' + str(perc) + '.png')
#             # Convert to RGB
#             im = Image.fromarray(slices).convert('RGB')
            
#             # save
#             im.save(title)
            
#     # Tweaking here since we have segmented files with 0% tumor
#     try:
#         create_transversal(aus_modalities, seg_img_data, save_path = tra_path) 
#     except:
#         # Map patient names 
#         index_patient = [i for i, v in enumerate(annotations_LGG.itertuples()) if v.CDE_ID == p_name]
        
#         # Get indices of tumor slices 
#         xmin = int(annotations_LGG.at[index_patient[0], 'xmin'])    
#         xmax = int(annotations_LGG.at[index_patient[0], 'xmax'])   
#         zmin = int(annotations_LGG.at[index_patient[0], 'zmin'])   
#         zmax = int(annotations_LGG.at[index_patient[0], 'zmax'])
        
#         patient_id = annotations_LGG.CDE_ID.map(str)
#         patient_id = patient_id.tolist()
        
        
#         create_transversal(aus_modalities, seg_img_data, save_path = tra_path, 
#                            from_excel = True, 
#                            tr_0 = xmin, 
#                            tr_1 = xmax)